# Pseudorandom number generators

Remember the one time pad:

In [ ]:
from random import randrange
import string

def vigenere_key_generator(secret_key_size: int) -> str:
    n = len(string.ascii_lowercase)
    secret_key = ''
    while len(secret_key) < secret_key_size:
        secret_key += string.ascii_lowercase[randrange(n)]
    return secret_key

def shift_letter(letter: str, shiftby: str, forward: bool=True) -> str:
    n = len(string.ascii_lowercase)
    
    letter_int = ord(letter) - 97
    shiftby_int = ord(shiftby) - 97
    
    if forward:
        return string.ascii_lowercase[(letter_int+shiftby_int)%n]
    else:
        return string.ascii_lowercase[(letter_int-shiftby_int)%n]
    
def vigenere_encrypt_decrypt(message: str, secret_key: str, encrypt:bool = True) -> str:
    key_len = len(secret_key)
    
    encoded = ''
    for i, letter in enumerate(message):
        if letter != " ":
            encoded += shift_letter(letter, secret_key[i%key_len], forward=encrypt)
        else:
            encoded += letter
    return encoded

In [ ]:
message = "helloworld"
secret_key = vigenere_key_generator(len(message))
ciphertext = vigenere_encrypt_decrypt(message, secret_key, encrypt=True)

print(f"message = {message}\nsecret_key = {secret_key}\nciphertext = {ciphertext}")

The one time pad in bits:

In [ ]:
from random import randrange

message = "01011101000101110101"
secret_key = ''.join([str(randrange(2)) for _ in range(len(message))])
ciphertext = ''.join([str((int(m)+int(s))%2) for m, s in zip(message, secret_key)])

In [ ]:
print(f"message:\n\t{message}")
print(f"secret_key\n\t{secret_key}")
print(f"ciphertext\n\t{ciphertext}")

If the secret_key is a sequence known by Alice and Bob and generated completely random, then the cryptosystem is perfectly secret. Problems for one time pad?

* key lenght has to be at least the same as message lenght
* the key has to be completely random 
* Alice and Bob have to meet each time to exchange the secret key

Let's design a system very close to pure random:

# Linear congruential generators

A linear congruential generator (LCG) is an algorithm that yields a sequence of pseudo-randomized numbers calculated with a discontinuous piecewise linear equation.

$$X_{n+1}=X_n*a+c \textrm{ (mod m)}$$.

There are several LCGs, in the example we take the one of numerical recipes correspoding to the values: a = 1664525, c = 1013904223 and m = $2^{32}$. The initial value $X_0$ is called the seed of the pseudoradom generator.

In [ ]:
a = 1664525
c = 1013904223
m = 2**32

def lcg(x0: int, a: int, c: int, m: int) -> int:
    return (x0*a+c)%m

In [ ]:
x0 = 433

l = []

xn = x0
for _ in range(100000):
    xn = lcg(xn, a, c, m)
    l.append(xn/m)

In [ ]:
import matplotlib.pyplot as plt

plt.clf()
fig = plt.figure(figsize=(16,7))
plt.hist(l, bins=50)
plt.show()

The good thing about pseudorandom generators is that you can reconstruct the pseudorandom numbers taking the same initial seed

In [ ]:
x0 = 53

xn = x0
for _ in range(10):
    xn = lcg(xn, a, c, m)
    print(xn)

In [ ]:
x0 = 53

xn = x0
for _ in range(10):
    xn = lcg(xn, a, c, m)
    print(xn)

This can be useful!. Alice and Bob can generate the same randomness if they keep the seed as a secret!. The secret is the Diffie-Hellman key they generated.

# Building PRNG function

We will use the common function ```randrange``` from random to create a binary pseudorandom generator.

In [ ]:
from random import seed

def PRNG(s: int, l: int) -> list:
    seed(s)
    
    prng = []
    while len(prng) < l:
        prng.append(randrange(2))
    return ''.join([str(n) for n in prng])

In [ ]:
s = 134323

# Alice's end
prng_alice = PRNG(s, 10)

# Bob's end
prng_bob = PRNG(s, 10)

print(prng_alice)
print(prng_bob)

# Better ways to generate random numbers in the computer

Our PRNG uses a seed as the source for the entropy for the random numbers. We can use **other seeds like environmental computer noise** (frequency of typing, temperature variation...). 

Using your system ```/dev/urandom```, e.g generate 10 random bytes (using MacOS). Open a terminal and run:

```head -c 10 /dev/urandom```

to see the output.

In [43]:
# calling the urandom function
import subprocess

command = "head -c 10 /dev/urandom"
process = subprocess.Popen(command.split(), stdout=subprocess.PIPE)
output, error = process.communicate()

print(f"Random bytes:\n\t{output}")

Random bytes:
	b'\xd2P\xbf\xbe\xb7D\xf1{\xc6\xc2'


In [44]:
# transform to integer bytes
int_bytes = [b for b in output]

print(f"Random bytes:\n\t{output}\nconverted integers:\n\t{int_bytes}")

Random bytes:
	b'\xd2P\xbf\xbe\xb7D\xf1{\xc6\xc2'
converted integers:
	[210, 80, 191, 190, 183, 68, 241, 123, 198, 194]


In [45]:
# transform to integer
int.from_bytes(output, "big")

993186517610735815345858

With the system random we can generate random bits of information. For instance:

In [213]:
# using secrets to acess the sytem random
import secrets

r = secrets.SystemRandom()
r.seed(23123)
bin(r.getrandbits(512))[2:]

'10101010000011100000010100010111111110000000001110101010011001111001000001000111101000101100011011111111110000110000110101100000001011101000101011101011100111000010010111110011001110101110011001111110111001101111100111100010100110100011111011100010011111101011111011100101011011000010101001110100010110001001010110111011010110001010011100010111101001111100110111110111011001011101011101001001110000110011000000111001110011010100111010111000010010100110110010010100000010111110001011000010100010111001000110100011'

## The quest for a perfect **random generator**

Using environmental computer noise is great as a source of entropy to generate random numbers, however the noise that Alice generates is differen than what Bob generates so they do not generate the same stream of bits. For now, we are forced to use pseudorandom generators based on a single seed, the secret key.

We need that:

* The probability to randomly generate each bit 1 or 0 is 0.5
* Alice and Bob generate the same exact stream of random bits

How can we do that?

Hopefully quantum physics comes to help. Quantum physics is intrinsecally random and therfore we can use it to generate purely random numbers. Also there are ways to correlate two systems (Alice and Bob) so that they generate the same stream of random numbers. This physics phenomenon is called [quantum entanglement](https://en.wikipedia.org/wiki/Quantum_entanglement) and is an active field of research in both experimental and theoretical physics.